## Analysis of a 2D Assembly System with 4 Degrees of Freedom Using Manual Construction of Vector Loops, Transformation Matrices, etc.

<div>
<img src="../Pictures/SchemaModeleJeuxTolerance.png" width="750"/>
</div>

### **Notebook Overview**
This notebook demonstrates the analysis of a 2D assembly system with four degrees of freedom. It involves manually constructing vector loops, transformation matrices, and defining compatibility equations and interface constraints. The process also includes uncertainty quantification and Monte Carlo analysis.

#### **Initialization and Imports**

In [4]:
import numpy as np
import sympy as sp
import openturns as ot
import matplotlib.pyplot as plt

from IPython.display import display, clear_output, HTML, IFrame
from time import time, sleep
from scipy.optimize import OptimizeResult, minimize, Bounds, LinearConstraint

import otaf

# Identity and 180° rotation matrix around z
I4 = otaf.I4()
J4 = otaf.J4()

#### **Definition of Nominal Dimensions**

In [5]:
### Different measures of our problem
X1 = 99.8  # Nominal Length of the male piece
X2 = 100.0  # Nominal Length of the female piece
X3 = 10.0  # Nominal width of the pieces
j = X2 - X1  # Nominal play between parts.
T = 0.2  # Tolerance interval for X1 and X2
t_ = T / 2

#### **Global Coordinate System (R0)**

In [6]:
R0 = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])
x_, y_, z_ = R0[0], R0[1], R0[2]

#### **Characteristic Points of Each Surface / Feature**

In [7]:
# Pièce 1 (male)
P1A0, P1A1, P1A2 = (
    np.array((0, X3 / 2, 0.0)),
    np.array((0, X3, 0.0)),
    np.array((0, 0, 0.0)),
)
P1B0, P1B1, P1B2 = (
    np.array((X1, X3 / 2, 0.0)),
    np.array((X1, X3, 0.0)),
    np.array((X1, 0, 0.0)),
)
P1C0, P1C1, P1C2 = (
    np.array((X1 / 2, 0, 0.0)),
    np.array((0, 0, 0.0)),
    np.array((X1, 0, 0.0)),
)

# Pièce 2 (femelle)  # On met les points à hM et pas hF pour qu'ils soient bien opposées! (Besoin??)
P2A0, P2A1, P2A2 = (
    np.array((0, X3 / 2, 0.0)),
    np.array((0, X3, 0.0)),
    np.array((0, 0, 0.0)),
)
P2B0, P2B1, P2B2 = (
    np.array((X2, X3 / 2, 0.0)),
    np.array((X2, X3, 0.0)),
    np.array((X2, 0, 0.0)),
)
P2C0, P2C1, P2C2 = (
    np.array((X2 / 2, 0, 0.0)),
    np.array((0, 0, 0.0)),
    np.array((X2, 0, 0.0)),
)

#### **Local Reference Frames for Each (Substitute) Surface**

In [8]:
# Male Part
RP1a = np.array([-1 * x_, -1 * y_, z_])
RP1b = R0
RP1c = np.array([-y_, x_, z_])

# Female Part
RP2a = R0
RP2b = np.array([-1 * x_, -1 * y_, z_])
RP2c = np.array([y_, -1 * x_, z_])

#### **Transformation Matrices and Deviation Matrices**

In [ ]:
# Transformation matrices for each contact point
TP1aA0, TP1aA1, TP1aA2 = (
    otaf.geometry.tfrt(RP1a, P1A0),
    otaf.geometry.tfrt(RP1a, P1A1),
    otaf.geometry.tfrt(RP1a, P1A2),
)
TP1bB0, TP1bB1, TP1bB2 = (
    otaf.geometry.tfrt(RP1b, P1B0),
    otaf.geometry.tfrt(RP1b, P1B1),
    otaf.geometry.tfrt(RP1b, P1B2),
)
TP1cC0, TP1cC1, TP1cC2 = (
    otaf.geometry.tfrt(RP1c, P1C0),
    otaf.geometry.tfrt(RP1c, P1C1),
    otaf.geometry.tfrt(RP1c, P1C2),
)

TP2aA0, TP2aA1, TP2aA2 = (
    otaf.geometry.tfrt(RP2a, P2A0),
    otaf.geometry.tfrt(RP2a, P2A1),
    otaf.geometry.tfrt(RP2a, P2A2),
)
TP2bB0, TP2bB1, TP2bB2 = (
    otaf.geometry.tfrt(RP2b, P2B0),
    otaf.geometry.tfrt(RP2b, P2B1),
    otaf.geometry.tfrt(RP2b, P2B2),
)
TP2cC0, TP2cC1, TP2cC2 = (
    otaf.geometry.tfrt(RP2c, P2C0),
    otaf.geometry.tfrt(RP2c, P2C1),
    otaf.geometry.tfrt(RP2c, P2C2),
)

In [ ]:
# Transformation matrices dictionary
TMD = {}
TMD["T1c1a"] = otaf.TransformationMatrix(initial=TP1cC0, final=TP1aA0)
TMD["T2a2c"] = otaf.TransformationMatrix(initial=TP2aA0, final=TP2cC0)
TMD["T1c1b"] = otaf.TransformationMatrix(initial=TP1cC0, final=TP1bB0)
TMD["T2b2c"] = otaf.TransformationMatrix(initial=TP2bB0, final=TP2cC0)

TMD["TP1aA1aA0"] = otaf.TransformationMatrix(initial=TP1aA1, final=TP1aA0)
TMD["TP2aA0aA1"] = otaf.TransformationMatrix(initial=TP2aA0, final=TP2aA1)
TMD["TP1aA2aA0"] = otaf.TransformationMatrix(initial=TP1aA2, final=TP1aA0)
TMD["TP2aA0aA2"] = otaf.TransformationMatrix(initial=TP2aA0, final=TP2aA2)

TMD["TP1bB1bB0"] = otaf.TransformationMatrix(initial=TP1bB1, final=TP1bB0)
TMD["TP2bB0bB1"] = otaf.TransformationMatrix(initial=TP2bB0, final=TP2bB1)
TMD["TP1bB2bB0"] = otaf.TransformationMatrix(initial=TP1bB2, final=TP1bB0)
TMD["TP2bB0bB2"] = otaf.TransformationMatrix(initial=TP2bB0, final=TP2bB2)

In [ ]:
# Deviation matrix with no defects
DI4 = otaf.DeviationMatrix(index=-1, translations="", rotations="")  # Pas de défauts

#### **Constructing Compatibility Loops**

##### **Loop 1: Compatibility (2c -> 1c -> 1a -> 2a)**

D2c2c -> GP2cC0P1cC0 -> J4 -> D1c1c -> T1c1a -> D1a1a -> GP1aA0P2aA0 -> J4 -> D2a2a -> T2a2c

In [ ]:
# Définissons les matrices :
D2c2c = D1c1c = D1a1a = D2a2a = DI4  # Pas de défauts

# GAP 0, contact plan-plan, contact bloqué par minimisation
GP2cC0P1cC0 = otaf.GapMatrix(index=0, translations_blocked="z", rotations_blocked="xy")
# GAP 1, contact plan-plan, tran x et rot z
GP1aA0P2aA0 = otaf.GapMatrix(index=1, translations_blocked="z", rotations_blocked="xy")

expa_1 = otaf.FirstOrderMatrixExpansion(
    [
        D2c2c,
        GP2cC0P1cC0,
        J4,
        D1c1c,
        TMD["T1c1a"],
        D1a1a,
        GP1aA0P2aA0,
        J4,
        D2a2a,
        TMD["T2a2c"],
    ]
).compute_first_order_expansion()
expa_1

##### **Loop 2: Compatibility (2c -> 1c -> 1b -> 2b)**

D2c2c -> GP2cC0P1cC0 -> J4 -> D1c1c -> T1c1b -> D1b1b -> GP1bB0P2bB0 -> J4 -> D2b2b -> T2b2c

In [ ]:
# Définissons les matrices :   # D2c2c -> GP2cC0P1cC0 -> D1c1c ->
D1b1b = otaf.DeviationMatrix(index=1, translations="x", rotations="z")  # Défauts plan
GP1bB0P2bB0 = otaf.GapMatrix(
    index=2, translations_blocked="z", rotations_blocked="xy"
)  # GAP 2, jeu plan
D2b2b = otaf.DeviationMatrix(
    index=2, translations="x", rotations="z", inverse=True
)  # Défauts plan #

expa_2 = otaf.FirstOrderMatrixExpansion(
    [
        D2c2c,
        GP2cC0P1cC0,
        J4,
        D1c1c,
        TMD["T1c1b"],
        D1b1b,
        GP1bB0P2bB0,
        J4,
        D2b2b,
        TMD["T2b2c"],
    ]
).compute_first_order_expansion()
expa_2

In [ ]:
    expr_compa_1 = otaf.common.get_relevant_expressions(expa_1)
    expr_compa_2 = otaf.common.get_relevant_expressions(expa_2)

compatibility_expressions = [*expr_compa_1, *expr_compa_2]

#### **Interface Constraints**

Define and compute interface constraints for each contact point.

##### **Boucle d'interface 1 côté A :**

In [ ]:
# GP1aA1P2aA1 = TP1aA1aA0 GP1aA0P2aA0 J4 TP2aA0aA1 J4
expa_f_1 = otaf.FirstOrderMatrixExpansion(
    [TMD["TP1aA1aA0"], GP1aA0P2aA0, J4, TMD["TP2aA0aA1"], J4]
).compute_first_order_expansion()
expa_f_1

In [ ]:
# GP1aA2P2aA2 = TP1aA2aA0 GP1aA0P2aA0 J4 TP2aA0aA2 J4
expa_f_2 = otaf.FirstOrderMatrixExpansion(
    [TMD["TP1aA2aA0"], GP1aA0P2aA0, J4, TMD["TP2aA0aA2"], J4]
).compute_first_order_expansion()
expa_f_2

##### **Boucle d'interface 2 côté B :**

In [ ]:
# GP1bB1P2bB1 = TP1bB1bB0 GP1bB0P2bB0 J4 TP2bB0bB1 J4
expa_f_3 = otaf.FirstOrderMatrixExpansion(
    [TMD["TP1bB1bB0"], GP1bB0P2bB0, J4, TMD["TP2bB0bB1"], J4]
).compute_first_order_expansion()
expa_f_3

In [ ]:
# GP1bB2P2bB2 = TP1bB2bB0 GP1bB0P2bB0 J4 TP2bB0bB2 J4
expa_f_4 = otaf.FirstOrderMatrixExpansion(
    [TMD["TP1bB2bB0"], GP1bB0P2bB0, J4, TMD["TP2bB0bB2"], J4]
).compute_first_order_expansion()
expa_f_4

In [ ]:
mask_matrix = sp.Matrix(
    np.array([[0, 0, 0, 1], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]])
)  # Only translation u should be positive so we only need its expression
expa_f_1 = expa_f_1.multiply_elementwise(mask_matrix)
expa_f_2 = expa_f_2.multiply_elementwise(mask_matrix)
expa_f_3 = expa_f_3.multiply_elementwise(mask_matrix)
expa_f_4 = expa_f_4.multiply_elementwise(mask_matrix)

In [ ]:
expr_fonc_1 = otaf.common.get_relevant_expressions(expa_f_1)
expr_fonc_2 = otaf.common.get_relevant_expressions(expa_f_2)
expr_fonc_3 = otaf.common.get_relevant_expressions(expa_f_3)
expr_fonc_4 = otaf.common.get_relevant_expressions(expa_f_4)

interface_constraints = [*expr_fonc_1, *expr_fonc_2, *expr_fonc_3, *expr_fonc_4]

In [ ]:
print("Compatibility equations:")
for i in range(len(compatibility_expressions)):
    display(compatibility_expressions[i])

print("Interface equations:")
for i in range(len(interface_constraints)):
    display(interface_constraints[i])

In [ ]:
SOCAM = otaf.SystemOfConstraintsAssemblyModel(
    compatibility_expressions, interface_constraints, verbose=2
)

SOCAM.embedOptimizationVariable()

C_opt = np.array([0, 1, 0, 0, 1, 0, 0, 0, 0])  # None
print(SOCAM.deviation_symbols)
print(SOCAM.gap_symbols)

In [ ]:
# D'abord les variances des lois si leur influence était unique (et nous a environ 95% de conformité)
T = 0.2  # Tolerance for X1 and X2. (95% conform)  (= t/2)
t_ = T / 2
Cm = 0.3
sigma_e_pos = T / (6 * Cm)

# Le défaut en orientation est piloté par une incertitude sur un angle. On suppose les angles petits << 1 rad
theta_max = T / X3
sigma_e_theta = (2*theta_max) / (6*Cm) 

In [ ]:
# Let's make our random vector With openturns
RandDeviationVect = otaf.uncertainty.get_composed_normal_defect_distribution(
    defect_names=SOCAM.deviation_symbols,
    sigma_dict = {"alpha":sigma_e_theta, 
                  "beta":sigma_e_theta,
                  "gamma":sigma_e_theta, 
                  "u":sigma_e_pos, 
                  "v":sigma_e_pos, 
                  "w":sigma_e_pos})

RandDeviationVect

#### **Monte Carlo Simulations for Probability of Failure**

##### Using the direct optimization model

In [ ]:
# Let's first generate a classic LHS design of expezriment of size 16.
D_lambd = len(SOCAM.deviation_symbols)
lambda_vect_unconditioned = ot.ComposedDistribution([ot.Uniform(0, 1)] * D_lambd)
lambda_vect_unconditioned.setDescription(list(map(str, SOCAM.deviation_symbols)))
N_lambda = 200
lambda_sample_unconditioned = otaf.uncertainty.generateLHSExperiment(lambda_vect_unconditioned ,N_lambda, 999)
#lambda_sample_unconditioned = lambda_sample_unconditioned_generator.generate()
lambda_sample_random = lambda_vect_unconditioned.getSample(N_lambda)
lambda_sample_conditioned = otaf.uncertainty.condition_lambda_sample(lambda_sample_random, squared_sum=True)
print(", ".join(map(str, SOCAM.gap_symbols)))
print("\n")
print(", ".join(map(str, SOCAM.deviation_symbols)))

#### Calculating over real model

In [ ]:
bounds = None
SEED_MC_PF = 6436431
SIZE_MC_PF = int(1e4) #1e4
optimizations_array = np.empty((N_lambda, SIZE_MC_PF), dtype=OptimizeResult)
failure_probabilities, successes, s_values, statuses = [], [], [], []
failed_optimization_points = []

start_time = time()  # Record the start time
for i in range(N_lambda):
    print(f"Doing iteration {i} of {N_lambda}")
    if i>0:
        print(f"Failure probability, Min: {min(failure_probabilities)}, / Max: {max(failure_probabilities)}")
        print(f"Failed {(1-successes).sum()} optimizations on { SIZE_MC_PF}")
        print("s_mean: ", s_values.mean().round(3), "s_min: ", np.nanmin(s_values).round(3), "s_max: ", np.nanmax(s_values).round(3))
        print("Statuses are:", np.unique(statuses))
    ot.RandomGenerator.SetSeed(SEED_MC_PF)
    deviation_samples = np.array(RandDeviationVect.getSample(SIZE_MC_PF)) * np.array(
        lambda_sample_conditioned[i]
    )
    optimizations = otaf.uncertainty.compute_gap_optimizations_on_sample(
            SOCAM,
            deviation_samples,
            bounds=bounds,
            n_cpu=-2,
            progress_bar=True,
        )
    successes = np.array([opt.success for opt in optimizations], dtype=bool)
    statuses = np.array([opt.status for opt in optimizations], dtype=int)
    
    if successes.sum() == 0:
        print("All optimizations failed")
        sleep(0.5)

    failed_optimization_points.append(deviation_samples[np.invert(successes), :])
    
    s_values = np.array([opt.fun for opt in optimizations], dtype=float)
    s_values = np.nan_to_num(s_values, nan=np.nanmax(s_values))*-1 # Cause the obj function C is -1*s and failed optimizations count as a negative s
    failure_probabilities.append(np.where(s_values < 0, 1, 0).mean())
    clear_output(wait=True)
print(f"Done {len (lambda_sample_conditioned)} experiments.")
print(f"Elapsed time: {time() - start_time:.3f} seconds.")
failed_optimization_points = np.vstack(failed_optimization_points)

X = otaf.uncertainty.find_best_worst_quantile(np.array(lambda_sample_conditioned), np.array(failure_probabilities), 0.1)
(best_5p_lambda, best_5p_res), (worst_5p_lambda, worst_5p_res) = X

In [ ]:
print("Lower probability of failure:", round(min(failure_probabilities) * 100, 4), "%")
print("Upper probability of failure:", round(max(failure_probabilities) * 100, 4), "%")
plt.hist(failure_probabilities)
plt.show()

In [ ]:
otaf.plotting.plot_best_worst_results(best_5p_res, worst_5p_res, figsize=(10,5))

variable_labels = [var for var in lambda_sample_conditioned.getDescription()]
otaf.plotting.plot_best_worst_input_data(best_5p_lambda, worst_5p_lambda, variable_labels, figsize=(10,5), labels=False)